# Deliver data for Analytics System 
- Tableau Dashboard
- KPIs

In [1]:
import sqlite3
import sys
from pathlib import Path

import pandas as pd
import yaml

In [2]:
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

# Base Data

In [3]:
# check dem system: 64 bit python? odb driver must be compatible with the python version
print(sys.version)
print(sys.maxsize > 2**32) # True for 64-bit, False for 32-bit

3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
True


In [4]:
# access db for tableau
# import pyodbc

# with pyodbc.connect(config['processed_data']['database_access']) as conn:
#     init_db(conn)

In [44]:
# export full data into csv file
db_path = Path(config['processed_data']['database_sqlite'])
query_path = Path(config['queries_path'])

with sqlite3.connect(db_path) as conn:
    
    with open(query_path.joinpath('create_views.sql'), 'r') as f:
        conn.executescript(f.read())

    with open(query_path.joinpath('select_data_for_tableau_dashboard.sql'), 'r') as f:
        # cur = conn.executescript(f.read())
        dashboard_data = pd.read_sql_query(f.read(), conn)
        dashboard_data.to_parquet(config['processed_data']['sample_parquet'])
        dashboard_data.to_csv(config['processed_data']['sample_csv'], index=False, encoding='utf-8')
dashboard_data.head()

,OrderDate,ProductID,CustomerID,ShipmentID,OrderReference,Quantity,Discount,Sales,Profit,DiscountAmount,...,ProductName,Category,SubCategory,ShipDate,ShipMode,PostalCode,City,Country,Region,State
0,2014-09-07 00:00:00,TEC-PH-10000000,DK-13375,0,CA-2014-100006,3,0.0,377.970,109.6113,0.000,...,AT&T EL51110 DECT,Technology,Phones,2014-09-13 00:00:00,Standard Class,10024,New York City,United States,East,New York
1,2014-10-19 00:00:00,TEC-PH-10000001,EH-14125,10,CA-2014-100867,6,0.2,321.552,20.0970,80.388,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2014-10-24 00:00:00,Standard Class,90712,Lakewood,United States,West,California
2,2014-11-21 00:00:00,TEC-PH-10000001,JK-15325,4328,US-2014-168501,5,0.2,267.960,16.7475,66.990,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2014-11-27 00:00:00,Standard Class,75220,Dallas,United States,Central,Texas
3,2015-06-16 00:00:00,TEC-PH-10000001,LC-16885,4459,US-2015-163825,2,0.0,133.980,33.4950,0.000,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2015-06-19 00:00:00,First Class,10009,New York City,United States,East,New York
4,2017-01-20 00:00:00,TEC-PH-10000001,TH-21100,4011,CA-2017-161809,3,0.2,160.776,10.0485,40.194,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2017-01-26 00:00:00,Standard Class,90045,Los Angeles,United States,West,California


In [45]:
%store dashboard_data

Stored 'dashboard_data' (DataFrame)


# KPI Data 

# EDA

In [40]:
dashboard_data.Sales.sum()

np.float64(2297200.8603)

In [24]:
dashboard_data.OrderDate = pd.to_datetime(dashboard_data.OrderDate)

In [11]:
dashboard_data.columns

Index(['OrderDate', 'ProductID', 'CustomerID', 'ShipmentID', 'OrderReference',
       'Quantity', 'Discount', 'Sales', 'Profit', 'DiscountAmount',
       'CustomerName', 'Segment', 'ProductName', 'Category', 'SubCategory',
       'ShipDate', 'ShipMode', 'PostalCode', 'City', 'Country', 'Region',
       'State'],
      dtype='object')

In [ ]:
{'Quantity':'sum', 'Discount':'sum', 'Sales':'sum', 'Profit':'sum', 'DiscountAmount':'sum'}

In [28]:
OrderDateGrouper = pd.Grouper(key='OrderDate', freq='YE')
dashboard_data[['Category', 'SubCategory', 'OrderDate', 'Quantity', 'Discount', 'Sales', 'Profit', 'DiscountAmount']].groupby(by=['Category', 'SubCategory', OrderDateGrouper]).sum().sort_index()#.sort_values(by='Profit')

Quantity  Discount        Sales  \
Category   SubCategory OrderDate                                     
Furniture  Bookcases   2014-12-31       153      7.61   20036.6776   
                       2015-12-31       242     13.94   38543.5725   
                       2016-12-31       197     10.27   26275.4665   
                       2017-12-31       276     16.32   30024.2797   
           Chairs      2014-12-31       540     22.20   77241.5760   
...                                     ...       ...          ...   
Technology Machines    2017-12-31       121      9.90   43544.6750   
           Phones      2014-12-31       684     26.20   77390.8060   
                       2015-12-31       697     28.00   68313.7020   
                       2016-12-31       818     35.00   78962.0300   
                       2017-12-31      1090     48.20  105340.5160   

                                       Profit  DiscountAmount  
Category   SubCategory OrderDate                               
Furniture  Bookcases   2014-12-31   -346.1707       5211.8024  
                       2015-12-31  -2755.2302      13623.5675  
                       2016-12-31    212.4710       6542.0335  
                       2017-12-31   -583.6261       7899.1503  
           Chairs      2014-12-31   6955.0470      15596.3840  
...                                       ...             ...  
Technology Machines    2017-12-31  -2869.2156      24338.8250  
           Phones      2014-12-31  11808.0053      13860.0540  
                       2015-12-31  10398.8074      12550.4780  
                       2016-12-31   9459.5929      17066.2000  
                       2017-12-31  12849.3250      22362.3340  

[68 rows x 5 columns]

In [36]:
dashboard_data[['Category', 'SubCategory', 'ProductName']].drop_duplicates().sort_values(by=['Category', 'SubCategory', 'ProductName'])

,Category,SubCategory,ProductName
9698,Furniture,Bookcases,"Atlantic Metals Mobile 2-Shelf Bookcases, Cust..."
9591,Furniture,Bookcases,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust..."
9585,Furniture,Bookcases,"Atlantic Metals Mobile 4-Shelf Bookcases, Cust..."
9673,Furniture,Bookcases,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust..."
9531,Furniture,Bookcases,Bestar Classic Bookcase
...,...,...,...
768,Technology,Phones,iKross Bluetooth Portable Keyboard + Cell Phon...
361,Technology,Phones,iOttie HLCRIO102 Car Mount
863,Technology,Phones,iOttie XL Car Mount
226,Technology,Phones,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...


In [41]:
dashboard_data.OrderReference.nunique()

5009